## Úvod
Tento notebook nemá složit jako výukový materiál. Jedná se jen o mé poznámky spojené s průzkumem balíčku Polars.
Výhody Polarsu by měly spočívat v rychlosti (koneckonců je napsán v Rustu), v možnosti by default pracovat na všech CPU jádrech a v schopnosti zpracovávat data větší než je velikost RAMky.

In [1]:
import polars as pl

Vytváření dataframu probíhá na první pohled stejně jako v pandách.

In [2]:
empty_frame = pl.DataFrame()
empty_frame

shape: (0, 0)
┌┐
╞╡
└┘

Všimněme si, že v zobrazené tabulce vidíme krom názvu sloupců i automaticky určený datový typ sloupce.

In [3]:
only_integers = [1,2,3,4,155]
only_strings = ["jedna", "dva", "tři", "čtyři", "pět"]
filled_dataframe = pl.DataFrame({
    "only_integers": only_integers, 
    "only_strings": only_strings
})
filled_dataframe

only_integers,only_strings
i64,str
1,"""jedna"""
2,"""dva"""
3,"""tři"""
4,"""čtyři"""
155,"""pět"""


Podstatnější rozdíly oproti balíčku pandas se objeví poměrně rychle. Pokud bychom bez jakýchkoli dalších úprav vložili do dataframu sloupec obsahující více datových typů,
např.
```
only_integers = [1,2,3,4,155]
only_strings = ["jedna", "dva", "tři", "čtyři", "pět"]
everything_possible = [1,2, "hodně", "málo", 1.23]
filled_dataframe = pl.DataFrame({
    "only_integers": only_integers, 
    "only_strings": only_strings, 
    "everything_possible": everything_possible
})
filled_dataframe
```
dostali bychom chybovou hlášku
```
TypeError: unexpected value while building Series of type Int64; found value of type String: "hodně"
```
Co s tím? Jednou z možností je vytvářet dataframe nikoli napřímo z listů, ale s pomocí series objektu, do kterého krom samotných dat vložíme i jejich datový typ. No a když použijeme datový typ *pl.Object*, je v datech akceptováno takřka cokoli. Nicméně člověk by měl být opatrný - koneckonců v dokumentaci se u [přehledu datových typů](https://docs.pola.rs/user-guide/concepts/data-types/overview/) uvádí, že podpora pro *Object* je omezená.

In [4]:
only_integers = [1,2,3,4,155]
only_strings = ["jedna", "dva", "tři", "čtyři", "pět"]
everything_possible = [1,2, "hodně", "málo", 1.23]
filled_dataframe = pl.DataFrame({
    "only_integers": pl.Series(only_integers, dtype=pl.Int16), 
    "only_strings": pl.Series(only_strings, dtype=pl.String), 
    "everything_possible": pl.Series(everything_possible, dtype=pl.Object)
})
filled_dataframe

only_integers,only_strings,everything_possible
i16,str,object
1,"""jedna""",1
2,"""dva""",2
3,"""tři""",hodně
4,"""čtyři""",málo
155,"""pět""",1.23


Zdůrazněme ještě, že s None problémy v tomto ohledu nejsou.

In [5]:
only_integers = [1,2,3,None,155]

short_dataframe = pl.DataFrame({
    "only_integers": pl.Series(only_integers, dtype=pl.Int16)
})
short_dataframe

only_integers
i16
1
2
3
null
155


Člověka napadá otázka, co se stane, když do malého datového typu zkusíme vložit velkou hodnotu (např. do int8 číslo větší než 128). Když bychom spustili následující kód
```
only_integers = [1,2,3,400,155]
filled_dataframe = pl.DataFrame({
    "only_integers": pl.Series(only_integers, dtype=pl.Int8), 
})
filled_dataframe
```
dostali bychom overflow chybu 
```
OverflowError: out of range integral type conversion attempted
```

Velikost tabulky určíme stejně jako v Pandách skrz *shape* a *len*. Stejně tak se lze podívat na první, poslední či náhodné řádky s pomocí *head*, *tail* a *sample*.

In [6]:
filled_dataframe.shape

(5, 3)

In [7]:
len(filled_dataframe)

5

In [8]:
filled_dataframe.head(2)

only_integers,only_strings,everything_possible
i16,str,object
1,"""jedna""",1
2,"""dva""",2


In [9]:
filled_dataframe.tail(2)

only_integers,only_strings,everything_possible
i16,str,object
4,"""čtyři""",málo
155,"""pět""",1.23


In [10]:
filled_dataframe.sample(2)

only_integers,only_strings,everything_possible
i16,str,object
4,"""čtyři""",málo
155,"""pět""",1.23


Rychlé statistické info typu průměr či maximum získáme díky metodě *describe*.

In [11]:
filled_dataframe.describe()

statistic,only_integers,only_strings,everything_possible
str,f64,str,str
"""count""",5.0,"""5""","""5"""
"""null_count""",0.0,"""0""","""0"""
"""mean""",33.0,null,null
"""std""",68.209237,null,null
"""min""",1.0,"""dva""",null
"""25%""",2.0,null,null
"""50%""",3.0,null,null
"""75%""",4.0,null,null
"""max""",155.0,"""čtyři""",null


Pokud chceme vidět unikátní hodnoty ve sloupci, použijeme metodu *unique* (ty selectovací kouzla jsou více diskutovány v kapitole níže):

In [12]:
only_integers = [1,2,3,None,155, 3, 2 ,3]

short_dataframe = pl.DataFrame({
    "only_integers": pl.Series(only_integers, dtype=pl.Int16)
})
short_dataframe.select(pl.col("only_integers").unique())

only_integers
i16
null
1
2
3
155


Pokud nám stačí jen počet unikátních hodnot, sáhneme po *n_unique*.

In [13]:
short_dataframe.select(pl.col("only_integers").n_unique())

only_integers
u32
5


A pokud potřebujeme počet výskytů každé unikátní hodnoty, je k dispozici metoda *value_counts*.

In [14]:
short_dataframe.select(pl.col("only_integers").value_counts())

only_integers
struct[2]
"{2,2}"
"{3,3}"
"{null,1}"
"{155,1}"
"{1,1}"


Pro výpočet korelace slouží metoda *corr*, pro její použití je ale potřeba mít nainstalovaný balíček *numpy*.

In [15]:
numbers_dataframe = pl.DataFrame({
    "first_col": pl.Series([10,20,30,40,50], dtype=pl.Int16),
    "second_col": pl.Series([50,40,30,20,10], dtype=pl.Int16),
    "third_col": pl.Series([25,40,12,78,51], dtype=pl.Int16)
})

numbers_dataframe.corr()

first_col,second_col,third_col
f64,f64,f64
1.0,-1.0,0.561754
-1.0,1.0,-0.561754
0.561754,-0.561754,1.0


## Manipulace se sloupci dataframu

Pro výběr sloupce překvapivě funguje pandí notace:

In [16]:
filled_dataframe["only_integers"]

only_integers
i16
1
2
3
4
155


Pokud chceme více sloupců, můžeme použít stejně jako v Pandách list, anebo jména sloupců napíšeme do vnějších hranatých závorek napřímo:

In [17]:
filled_dataframe[["only_integers", "only_strings"]]

only_integers,only_strings
i16,str
1,"""jedna"""
2,"""dva"""
3,"""tři"""
4,"""čtyři"""
155,"""pět"""


In [18]:
filled_dataframe["only_integers", "only_strings"]

only_integers,only_strings
i16,str
1,"""jedna"""
2,"""dva"""
3,"""tři"""
4,"""čtyři"""
155,"""pět"""


V dokumentaci Polarsu se nicméně ukazuje použití dataframové metody *select*, které podhodíme polarsí funkci *col* s chtěnými sloupci. Hádám, že to asi bude efektivnější než výše použitá "indexová" metoda.

In [19]:
filled_dataframe.select(pl.col("only_integers", "only_strings"))

only_integers,only_strings
i16,str
1,"""jedna"""
2,"""dva"""
3,"""tři"""
4,"""čtyři"""
155,"""pět"""


Kdyby to bylo z nějakého důvodu potřeba, tak aplikací hvězdičky přikážeme zobrazit všechny sloupce:

In [20]:
filled_dataframe.select(pl.col("*"))

only_integers,only_strings,everything_possible
i16,str,object
1,"""jedna""",1
2,"""dva""",2
3,"""tři""",hodně
4,"""čtyři""",málo
155,"""pět""",1.23


Přidávání nových sloupců skrz index, tj. operace
```
filled_dataframe["new_col_with_1"] = 1
```
už povolená není - člověk obdrží chybu
```
TypeError: DataFrame object does not support `Series` assignment by index
```
Musíme tak použít *with_columns* metodu. 

In [21]:
filled_dataframe = filled_dataframe.with_columns(
    1
)
filled_dataframe

only_integers,only_strings,everything_possible,literal
i16,str,object,i32
1,"""jedna""",1,1
2,"""dva""",2,1
3,"""tři""",hodně,1
4,"""čtyři""",málo,1
155,"""pět""",1.23,1


No, asi bychom rádi sloupci dali nějaké rozumné jméno. To provedeme skrze metodu *alias*. Jenže ta pro integer neexistuje. Proto musíme jedničku obalit do *pl.lit*. 

In [22]:
filled_dataframe = filled_dataframe.with_columns(
    pl.lit(1).alias("new_col_with_1")
)
filled_dataframe

only_integers,only_strings,everything_possible,literal,new_col_with_1
i16,str,object,i32,i32
1,"""jedna""",1,1,1
2,"""dva""",2,1,1
3,"""tři""",hodně,1,1
4,"""čtyři""",málo,1,1
155,"""pět""",1.23,1,1


Vytvoření nových sloupců s pomocí sloupců původních vypadá podobně (bacha - pokud bychom u jména sloupce "only_strings" zapomenuli na uvozovky, pochopí to polars jako svého druhu list s chtěnými sloupci a pak spadne na to, že hned prnví hodota - řetězec "jedna" - žádný sloupec neoznačuje):

In [23]:
filled_dataframe = filled_dataframe.with_columns(
    (pl.col("only_strings") + " " + pl.col("only_strings")).alias("double_string")
)
filled_dataframe

only_integers,only_strings,everything_possible,literal,new_col_with_1,double_string
i16,str,object,i32,i32,str
1,"""jedna""",1,1,1,"""jedna jedna"""
2,"""dva""",2,1,1,"""dva dva"""
3,"""tři""",hodně,1,1,"""tři tři"""
4,"""čtyři""",málo,1,1,"""čtyři čtyři"""
155,"""pět""",1.23,1,1,"""pět pět"""


Přeuspořádání sloupců realizujeme selectem:

In [24]:
filled_dataframe.select(["literal", "new_col_with_1", "double_string", "only_integers", "only_strings", "everything_possible"])

literal,new_col_with_1,double_string,only_integers,only_strings,everything_possible
i32,i32,str,i16,str,object
1,1,"""jedna jedna""",1,"""jedna""",1
1,1,"""dva dva""",2,"""dva""",2
1,1,"""tři tři""",3,"""tři""",hodně
1,1,"""čtyři čtyři""",4,"""čtyři""",málo
1,1,"""pět pět""",155,"""pět""",1.23


Sloupců se zbavíme s pomocí metody *drop*:

In [25]:
filled_dataframe = filled_dataframe.drop(["literal", "new_col_with_1", "double_string"])
filled_dataframe

only_integers,only_strings,everything_possible
i16,str,object
1,"""jedna""",1
2,"""dva""",2
3,"""tři""",hodně
4,"""čtyři""",málo
155,"""pět""",1.23


Nakonec pokud bychom k něčemu potřebovali list jmen sloupců dataframe, přijde nám vhod atribut dataframu *columns*.

In [26]:
filled_dataframe.columns

['only_integers', 'only_strings', 'everything_possible']

## Manipulace s řádky dataframu

Pro získání řádky dataframu nepoužijeme (i)loc, nýbrž napřímo index. Zdá se, že nic jako pojmonované idnexy v Polarsu neexistují.

In [27]:
filled_dataframe[1]

only_integers,only_strings,everything_possible
i16,str,object
2,"""dva""",2


Pokud se pokusíme použít příliš velký (neexistující) index, nedostaneme chybu, nýbrž prázdný výsledek:

In [28]:
filled_dataframe[100]

only_integers,only_strings,everything_possible
i16,str,object


Zdá se, že explicitní vyhození řádku v Polars neexistuje. Bude se tudíž muset použít filtrování. To realizujeme s pomocí metody filter, do které umístíme podmínku.  

In [29]:
filled_dataframe.filter(
    pl.col("only_strings") == "dva"
)

only_integers,only_strings,everything_possible
i16,str,object
2,"""dva""",2


Concatování funguje dle očekávání s použítím funkce *concat*, která přebírá list dataframů. 

In [30]:
pl.concat([filled_dataframe, filled_dataframe])

only_integers,only_strings,everything_possible
i16,str,object
1,"""jedna""",1
2,"""dva""",2
3,"""tři""",hodně
4,"""čtyři""",málo
155,"""pět""",1.23
1,"""jedna""",1
2,"""dva""",2
3,"""tři""",hodně
4,"""čtyři""",málo


Nicméně mám podezření, že více podporovaná bude metoda *vstack*.

In [31]:
filled_dataframe.vstack(filled_dataframe)

only_integers,only_strings,everything_possible
i16,str,object
1,"""jedna""",1
2,"""dva""",2
3,"""tři""",hodně
4,"""čtyři""",málo
155,"""pět""",1.23
1,"""jedna""",1
2,"""dva""",2
3,"""tři""",hodně
4,"""čtyři""",málo


Zdá se, že metoda apply v Polars existovala jen [ve starých verzích](https://docs.pola.rs/api/python/version/0.18/reference/dataframe/api/polars.DataFrame.apply.html) - v dokumentaci pro nové verze už k nalezení není a ukázkový příklad nefunguje.

Pro změnu jedné buňky dataframu lze použít následující postup. Ten nicméně podle [diskuse na Githubu](https://github.com/pola-rs/polars/issues/5973) není efektivní.

In [32]:
filled_dataframe[3, "only_integers"] = 400
filled_dataframe

only_integers,only_strings,everything_possible
i16,str,object
1,"""jedna""",1
2,"""dva""",2
3,"""tři""",hodně
400,"""čtyři""",málo
155,"""pět""",1.23


## SQL-like dotazy v Polarsu

Napřed si připravme tabulky:

In [33]:
osoby = pl.DataFrame({
    "id" : [100,200,300,400,500],
    "krestni_jmeno" : ["Victor", "Mary", "Johann", "Albert", "William"],
    "prijmeni" : ["Hugo", "Shelly", "Geothe", "Camus", "Shakespear"],
    "vek" : [25,30,75,None,38],
    "pohlavi" : ["M", "F", "M", "M", "M"]
})
osoby

id,krestni_jmeno,prijmeni,vek,pohlavi
i64,str,str,i64,str
100,"""Victor""","""Hugo""",25,"""M"""
200,"""Mary""","""Shelly""",30,"""F"""
300,"""Johann""","""Geothe""",75,"""M"""
400,"""Albert""","""Camus""",null,"""M"""
500,"""William""","""Shakespear""",38,"""M"""


In [34]:
prodane_knihy = pl.DataFrame({
    "id" : [100,200,300,400],
    "pocet_knih" : [19,25,12,31]
})
prodane_knihy

id,pocet_knih
i64,i64
100,19
200,25
300,12
400,31


Klasický select
```
select krestni_jmeno, prijmeni, vek from osoby;
```
realizujeme v polarsu s pomocí metody *select*:

In [35]:
osoby.select(["krestni_jmeno", "prijmeni", "vek"])

krestni_jmeno,prijmeni,vek
str,str,i64
"""Victor""","""Hugo""",25
"""Mary""","""Shelly""",30
"""Johann""","""Geothe""",75
"""Albert""","""Camus""",null
"""William""","""Shakespear""",38


Pokud bychom chtěli jen dva řádky, přidáme *head*:

In [36]:
osoby.select(["krestni_jmeno", "prijmeni", "vek"]).head(2)

krestni_jmeno,prijmeni,vek
str,str,i64
"""Victor""","""Hugo""",25
"""Mary""","""Shelly""",30


Pro WHERE podmínku ve stylu
```
select * from osoby where prijmeni = 'Geothe';
```
aplikujeme metody *filter*:

In [37]:
osoby.filter(
    pl.col("prijmeni") == "Geothe"
)

id,krestni_jmeno,prijmeni,vek,pohlavi
i64,str,str,i64,str
300,"""Johann""","""Geothe""",75,"""M"""


Pro více podmínek použijeme jejich spojení s pomocí & či | jako v pandách. 
```
select * from osoby where prijmeni = 'Geothe' and krestni_jmeno = "Johann";
```

In [38]:
osoby.filter(
    (pl.col("prijmeni") == "Geothe")
    & (pl.col("krestni_jmeno") == "Johann")
)

id,krestni_jmeno,prijmeni,vek,pohlavi
i64,str,str,i64,str
300,"""Johann""","""Geothe""",75,"""M"""


V případě, že chceme omezení současně na řádky a současně na sloupce, si musíme narozdíl od pand dát pozor na pořadí metod *select* a *filter*. Pokud by totiž *select* byl první, už by nemusely existovat sloupce, které *filter* potřebuje.
```
select id from osoby where prijmeni = 'Geothe' and krestni_jmeno = "Johann";
```

In [39]:
osoby.filter(
    (pl.col("prijmeni") == "Geothe")
    & (pl.col("krestni_jmeno") == "Johann")
).select(["id"])

id
i64
300


Jak v Polarsu ošetřit existenci None? Narozdíl od pandího *is_na* použijeme metodu *is_null*. V Polarsu na rozdíl od Pand neexistuje podle základního datového typu více None hodnot - vše je "null".
```
select * osoby where vek is NULL;
```

In [40]:
osoby.filter(
    pl.col("vek").is_null()
)

id,krestni_jmeno,prijmeni,vek,pohlavi
i64,str,str,i64,str
400,"""Albert""","""Camus""",null,"""M"""


Oproti tomu pro ekvivalent sqlkovského like máme k dispozici stejně jako v Pandách *str.contains*.
```
select * osoby where prijmeni like '%eoth%';
```

In [41]:
osoby.filter(
    pl.col("prijmeni").str.contains("eoth")
)

id,krestni_jmeno,prijmeni,vek,pohlavi
i64,str,str,i64,str
300,"""Johann""","""Geothe""",75,"""M"""


Stejně jako v Pandách slouží k negaci podmínky vlnovka.

In [42]:
osoby.filter(
    ~pl.col("prijmeni").str.contains("eoth")
)

id,krestni_jmeno,prijmeni,vek,pohlavi
i64,str,str,i64,str
100,"""Victor""","""Hugo""",25,"""M"""
200,"""Mary""","""Shelly""",30,"""F"""
400,"""Albert""","""Camus""",null,"""M"""
500,"""William""","""Shakespear""",38,"""M"""


Tabulku seřazíme metodou *sort*. Defaultně je řazení rostoucí, což lze změnit s přiřazením True do parametru *descending*.

In [43]:
osoby.sort(by="vek", descending=True)

id,krestni_jmeno,prijmeni,vek,pohlavi
i64,str,str,i64,str
400,"""Albert""","""Camus""",null,"""M"""
300,"""Johann""","""Geothe""",75,"""M"""
500,"""William""","""Shakespear""",38,"""M"""
200,"""Mary""","""Shelly""",30,"""F"""
100,"""Victor""","""Hugo""",25,"""M"""


Grupování provedeme s metodou *group_by* (bacha - oproti pandímu *groupby* tu je navíc podtržítko).
```
select pohlavi, count(*), avg(vek) from osoby group by pohlavi;
```

In [44]:
osoby.group_by("pohlavi").agg(
    pl.col("vek").mean().alias("prumerny_vek"),
    pl.col("pohlavi").count().alias("pocet"),
)

pohlavi,prumerny_vek,pocet
str,f64,u32
"""F""",30.0,1
"""M""",46.0,4


Na joinování slouží dataframová metoda *join*. Pokud joinovací sloupec (resp. sloupce) nese stejné jméno v obou tabulkách, použijeme parametr *on*, jinak *left_on* a *right_on*. Typ joinu bude specifikován parametrem *how*.
```
select * 
from 
 osoby os1 
inner join 
 prodane_knihy pk1 
   on os1.id = pk1.id;
```

In [45]:
osoby.join(prodane_knihy, on="id", how="left")

id,krestni_jmeno,prijmeni,vek,pohlavi,pocet_knih
i64,str,str,i64,str,i64
100,"""Victor""","""Hugo""",25,"""M""",19
200,"""Mary""","""Shelly""",30,"""F""",25
300,"""Johann""","""Geothe""",75,"""M""",12
400,"""Albert""","""Camus""",null,"""M""",31
500,"""William""","""Shakespear""",38,"""M""",null


Joinování na nerovnost se musí stejně jako v pandách ošetřit cross joinem a podmínkou.
```
select * 
from osoby oso1
inner join osoby oso2
on oso1.id < oso2.id;
```

In [46]:
crossed_join = osoby.join(osoby, on="id", how="cross")
crossed_join.filter(pl.col("id") < pl.col("id_right"))

id,krestni_jmeno,prijmeni,vek,pohlavi,id_right,krestni_jmeno_right,prijmeni_right,vek_right,pohlavi_right
i64,str,str,i64,str,i64,str,str,i64,str
100,"""Victor""","""Hugo""",25,"""M""",200,"""Mary""","""Shelly""",30,"""F"""
100,"""Victor""","""Hugo""",25,"""M""",300,"""Johann""","""Geothe""",75,"""M"""
100,"""Victor""","""Hugo""",25,"""M""",400,"""Albert""","""Camus""",null,"""M"""
100,"""Victor""","""Hugo""",25,"""M""",500,"""William""","""Shakespear""",38,"""M"""
200,"""Mary""","""Shelly""",30,"""F""",300,"""Johann""","""Geothe""",75,"""M"""
200,"""Mary""","""Shelly""",30,"""F""",400,"""Albert""","""Camus""",null,"""M"""
200,"""Mary""","""Shelly""",30,"""F""",500,"""William""","""Shakespear""",38,"""M"""
300,"""Johann""","""Geothe""",75,"""M""",400,"""Albert""","""Camus""",null,"""M"""
300,"""Johann""","""Geothe""",75,"""M""",500,"""William""","""Shakespear""",38,"""M"""


Pro union all použijeme metodu *vstack*.
```
select * from osoby 
union all
select * from osoby;
```

In [47]:
osoby.vstack(osoby)

id,krestni_jmeno,prijmeni,vek,pohlavi
i64,str,str,i64,str
100,"""Victor""","""Hugo""",25,"""M"""
200,"""Mary""","""Shelly""",30,"""F"""
300,"""Johann""","""Geothe""",75,"""M"""
400,"""Albert""","""Camus""",null,"""M"""
500,"""William""","""Shakespear""",38,"""M"""
100,"""Victor""","""Hugo""",25,"""M"""
200,"""Mary""","""Shelly""",30,"""F"""
300,"""Johann""","""Geothe""",75,"""M"""
400,"""Albert""","""Camus""",null,"""M"""


Pokud ale potřebujeme union, tj. nechceme mít duplicitní řádky, musíme navíc použít metodu *unique*.
```
select * from osoby 
union
select * from osoby;
```

In [48]:
osoby.vstack(osoby).unique()

id,krestni_jmeno,prijmeni,vek,pohlavi
i64,str,str,i64,str
300,"""Johann""","""Geothe""",75,"""M"""
400,"""Albert""","""Camus""",null,"""M"""
500,"""William""","""Shakespear""",38,"""M"""
100,"""Victor""","""Hugo""",25,"""M"""
200,"""Mary""","""Shelly""",30,"""F"""


Výše jsme řešili ekvivalenty SQL příkazů, nicméně SQLka lze na dataframy vypustit i napřímo (účel metody collect viz níže v kapitole o lazy přístupu).

In [49]:
lazy_sql_result = pl.SQLContext(frame=osoby).execute(
    "SELECT * FROM frame WHERE vek is null"
)
lazy_sql_result.collect()

id,krestni_jmeno,prijmeni,vek,pohlavi
i64,str,str,i64,str
400,"""Albert""","""Camus""",null,"""M"""


## Nahrávání a ukládání dat



Pro načtení csv souboru použijeme funkci *read_csv*

In [50]:
iris_dataset = pl.read_csv(
    "iris.data", separator=",", has_header=False,
    new_columns=["col1", "col2", "col3", "col4", "target"]
)
iris_dataset.head()

col1,col2,col3,col4,target
f64,f64,f64,f64,str
5.1,3.5,1.4,0.2,"""Iris-setosa"""
4.9,3.0,1.4,0.2,"""Iris-setosa"""
4.7,3.2,1.3,0.2,"""Iris-setosa"""
4.6,3.1,1.5,0.2,"""Iris-setosa"""
5.0,3.6,1.4,0.2,"""Iris-setosa"""


Pro načítání souboru po částech slouží *read_csv_batched*, pro tvorbu lazy dataframu *scan_csv*. Existují i funkce na načtení excelu či ods souborů. Lze nalézt i metodu *read_database*, která dle popisu funguje podobně jako pandí *read_sql*.

Zápis do souboru realizujeme skrze metodu *write_csv*.

In [51]:
iris_dataset.write_csv("ukladani_souboru.csv", separator="|")

## Lazy přístup

V defaultním módu se Polars chová podobně jako Pandas, tj. po odpálení každé řádky se fakticky provede něco s daty. Polars nicméně podporuje i lazy přístup, kdy spuštění příkazu povede pouze k přidání další položky do query plánu. Ten se spustí až po provolání jedné z mála speciálních metod. A k čemu je to dobré? Jednak může položky plánu automat optimalizovat, jednak lze v kombinaci se streamováním zpracovávat i datasety větší než velikost RAMky.

Dataframe může být lazy už od svého nahrání ze souboru. Druhou možností je vzít normální dataframe a na lazy ho s pomocí metody *lazy* ztransformovat.
Příkazy lze na sebe vázat tečkovou notací anebo normálně brát řádek po řádku, nehraje to roli.

In [52]:
lazy_df_from_beginning = (
    pl.scan_csv("iris.data", has_header=False)
    .with_columns(pl.col("column_5").str.to_uppercase())
    .filter(pl.col("column_1") > 3)
)

In [53]:
normal_df = pl.read_csv(
    "iris.data", separator=",", has_header=False,
    new_columns=["col1", "col2", "col3", "col4", "target"]
)
lazy_df_transf = normal_df.lazy()

Pro proběhnutí celého plánu musíme provolat metodu *collect*.

In [54]:
lazy_df_from_beginning = lazy_df_from_beginning.head()
lazy_df_from_beginning.collect()

column_1,column_2,column_3,column_4,column_5
f64,f64,f64,f64,str
5.1,3.5,1.4,0.2,"""IRIS-SETOSA"""
4.9,3.0,1.4,0.2,"""IRIS-SETOSA"""
4.7,3.2,1.3,0.2,"""IRIS-SETOSA"""
4.6,3.1,1.5,0.2,"""IRIS-SETOSA"""
5.0,3.6,1.4,0.2,"""IRIS-SETOSA"""


Všimněme si, že u transformovaného dataframu se ztratily jména sloupců a byly nahrazeny tím samým, co má první lazy dataframe - column_X, kde X je rostoucí celé číslo začínající jedničkou.

In [55]:
lazy_df_transf = lazy_df_from_beginning.head()
lazy_df_transf.collect()

column_1,column_2,column_3,column_4,column_5
f64,f64,f64,f64,str
5.1,3.5,1.4,0.2,"""IRIS-SETOSA"""
4.9,3.0,1.4,0.2,"""IRIS-SETOSA"""
4.7,3.2,1.3,0.2,"""IRIS-SETOSA"""
4.6,3.1,1.5,0.2,"""IRIS-SETOSA"""
5.0,3.6,1.4,0.2,"""IRIS-SETOSA"""


Pokud chceme pracovat s daty většími než je kapacita RAMky, musíme do metody *collect* přidat parametr *streaming* s hodnotou True.

In [56]:
lazy_df_from_beginning.collect(streaming=True)

column_1,column_2,column_3,column_4,column_5
f64,f64,f64,f64,str
5.1,3.5,1.4,0.2,"""IRIS-SETOSA"""
4.9,3.0,1.4,0.2,"""IRIS-SETOSA"""
4.7,3.2,1.3,0.2,"""IRIS-SETOSA"""
4.6,3.1,1.5,0.2,"""IRIS-SETOSA"""
5.0,3.6,1.4,0.2,"""IRIS-SETOSA"""


V dokumentaci se píše, že pokud při vývoji kódu nechceme provádět operace na celých datech (každá iterace by trvala příliš dlouho), ale jen na pár záznamech, aplikujeme namísto metody *collect* metodu *fetch*. Do ní umístíme parametr *n_rows* s počtem řádků pro daný běh. Když ale kód spustíme, uvidíme deprecation error s tím, že se má použít *collect*, před kterým bude *head*...

In [57]:
lazy_df_from_beginning.fetch(n_rows=3)

C:\Users\worker\AppData\Local\Temp\ipykernel_7472\575994917.py:1: DeprecationWarning: `LazyFrame.fetch` is deprecated. `LazyFrame.fetch` is deprecated; use `LazyFrame.collect` instead, in conjunction with a call to `head`.
  lazy_df_from_beginning.fetch(n_rows=3)


column_1,column_2,column_3,column_4,column_5
f64,f64,f64,f64,str
5.1,3.5,1.4,0.2,"""IRIS-SETOSA"""
4.9,3.0,1.4,0.2,"""IRIS-SETOSA"""
4.7,3.2,1.3,0.2,"""IRIS-SETOSA"""


Zobrazení exekučních plánů provedeme s pomocí metody *explain*. U ní je nejdůležitější si uvědomit, že se její výstup musí číst odspodu.

In [58]:
print(
    lazy_df_from_beginning.explain(optimized=False)
)

SLICE[offset: 0, len: 5]
  FILTER [(col("column_1")) > (3.0)] FROM
     WITH_COLUMNS:
     [col("column_5").str.uppercase()] 
      Csv SCAN [iris.data]
      PROJECT */5 COLUMNS


In [59]:
print(
    lazy_df_from_beginning.explain(optimized=True)
)

 WITH_COLUMNS:
 [col("column_5").str.uppercase()] 
  SLICE[offset: 0, len: 5]
    Csv SCAN [iris.data]
    PROJECT */5 COLUMNS
    SELECTION: [(col("column_1")) > (3.0)]


## Testy

Z hlediska jednotkových testů můžeme použít podobně jako v pandách funkci *assert_frame_equal* a to jak na normální framy, tak na lazy framy.

In [60]:
from polars.testing import assert_frame_equal

In [61]:
only_integers = [1,2,3,4,155]
only_strings = ["jedna", "dva", "tři", "čtyři", "pět"]

first_dataframe = pl.DataFrame({
    "only_integers": only_integers, 
    "only_strings": only_strings
})

second_dataframe = pl.DataFrame({
    "only_integers": only_integers, 
    "only_strings": only_strings
})

assert_frame_equal(first_dataframe, second_dataframe)

In [62]:
only_integers = [1,2,3,4,155]
only_strings = ["jedna", "dva", "tři", "čtyři", "pět"]

first_dataframe = pl.DataFrame({
    "only_integers": only_integers, 
    "only_strings": only_strings
})

second_dataframe = pl.DataFrame({
    "only_integers": only_integers, 
    "only_strings": only_integers
})

assert_frame_equal(first_dataframe, second_dataframe)

AssertionError: DataFrames are different (dtypes do not match)
[left]:  {'only_integers': Int64, 'only_strings': String}
[right]: {'only_integers': Int64, 'only_strings': Int64}

In [63]:
only_integers = [1,2,3,4,155]
only_strings = ["jedna", "dva", "tři", "čtyři", "pět"]

first_dataframe = pl.DataFrame({
    "only_integers": only_integers, 
    "only_strings": only_strings
}).lazy()

second_dataframe = pl.DataFrame({
    "only_integers": only_integers, 
    "only_strings": only_strings
}).lazy()

assert_frame_equal(first_dataframe, second_dataframe)

In [64]:
only_integers = [1,2,3,4,155]
only_strings = ["jedna", "dva", "tři", "čtyři", "pět"]
only_strings2 = ["jedna", "dva", "tři", "čtyři", "padesát"]

first_dataframe = pl.DataFrame({
    "only_integers": only_integers, 
    "only_strings": only_strings
}).lazy()

second_dataframe = pl.DataFrame({
    "only_integers": only_integers, 
    "only_strings": only_strings2
}).lazy()

assert_frame_equal(first_dataframe, second_dataframe)

AssertionError: LazyFrames are different (value mismatch for column 'only_strings')
[left]:  ['jedna', 'dva', 'tři', 'čtyři', 'pět']
[right]: ['jedna', 'dva', 'tři', 'čtyři', 'padesát']